In [1]:
import modin.pandas as mpd
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


## Prepare dataset

In [ ]:
df = pd.read_csv('internet_service_churn.csv')
newdf = pd.DataFrame(np.repeat(df.values, 100, axis=0))
newdf.columns = df.columns
newdf.to_csv('newdf.csv',index=False)

## Read Dataset and benchmarking

In [3]:
FILENAME = 'newdf.csv'

In [4]:
%%time
pdf = pd.read_csv(FILENAME)

CPU times: user 9.15 s, sys: 2.46 s, total: 11.6 s
Wall time: 17.2 s


In [6]:
%%time
mdf = mpd.read_csv(FILENAME)

CPU times: user 589 ms, sys: 444 ms, total: 1.03 s
Wall time: 12.8 s


In [26]:
pdf.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15.0,1.0,0.0,11.95,25.0,0.14,0.0,8.4,2.3,0.0,0.0
1,15.0,1.0,0.0,11.95,25.0,0.14,0.0,8.4,2.3,0.0,0.0
2,15.0,1.0,0.0,11.95,25.0,0.14,0.0,8.4,2.3,0.0,0.0
3,15.0,1.0,0.0,11.95,25.0,0.14,0.0,8.4,2.3,0.0,0.0
4,15.0,1.0,0.0,11.95,25.0,0.14,0.0,8.4,2.3,0.0,0.0


In [34]:
pdf.shape

(7227400, 11)

## write operations

In [35]:
%%time
pdf.to_csv('pdf.csv',index=False)

CPU times: user 1min 7s, sys: 1.62 s, total: 1min 9s
Wall time: 1min 16s


In [36]:
%%time
mdf.to_csv('mdf.csv',index=False)

CPU times: user 1.13 s, sys: 161 ms, total: 1.29 s
Wall time: 1min 11s


## ml modeling

###### Tried xgbooset_modin but couldnt run it so settling with RF and KNN

In [38]:
# !pip install ray boto3
# !ray up modin_xgboost.yaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Provided cluster configuration file (modin_xgboost.yaml) does not exist


In [42]:
# from modin.experimental.sklearn.model_selection import train_test_split
# from xgboost_ray import RayDMatrix, RayParams, train, predict

In [41]:
# !pip install xgboost_ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 KB 3.3 MB/s eta 0:00:00


#### Dropping NA (instead of treating them properly as the focus is to run and benchmark)

In [45]:
pdf = pdf.dropna()

In [46]:
pdf.columns

Index(['id', 'is_tv_subscriber', 'is_movie_package_subscriber',
       'subscription_age', 'bill_avg', 'reamining_contract',
       'service_failure_count', 'download_avg', 'upload_avg',
       'download_over_limit', 'churn'],
      dtype='object')

In [7]:

from sklearn.preprocessing import StandardScaler

In [54]:
X2 = StandardScaler().fit_transform(pdf[['is_tv_subscriber', 'is_movie_package_subscriber',
       'subscription_age', 'bill_avg', 'reamining_contract',
       'service_failure_count', 'download_avg', 'upload_avg',
       'download_over_limit']])

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

### dropping duplicate to make sure we dont have the same records in train and test dataset

In [6]:
pdf = pdf.drop_duplicates()


#### split data

In [7]:
x_train, x_test, y_train, y_test = train_test_split(pdf[['is_tv_subscriber', 'is_movie_package_subscriber','subscription_age', 'bill_avg', 'reamining_contract','service_failure_count', 'download_avg', 'upload_avg','download_over_limit']], pdf[['churn']], test_size = 0.30, random_state = 0)

In [8]:
x_train_scaled = pd.DataFrame(np.repeat(x_train.values, 100, axis=0))
x_train_scaled.columns = x_train.columns
y_train_scaled = pd.DataFrame(np.repeat(y_train.values, 100, axis=0))
y_train_scaled.columns = y_train.columns


In [9]:
x_test_scaled = pd.DataFrame(np.repeat(x_test.values, 100, axis=0))
x_test_scaled.columns = x_test.columns
y_test_scaled = pd.DataFrame(np.repeat(y_test.values, 100, axis=0))
y_test_scaled.columns = y_test.columns


In [10]:
from sklearn.ensemble import RandomForestClassifier

In [79]:
%%time
clf = RandomForestClassifier(n_estimators=5, random_state=0)
clf.fit(x_train_scaled, y_train_scaled)

CPU times: user 31.4 s, sys: 828 ms, total: 32.2 s
Wall time: 39.2 s


RandomForestClassifier(n_estimators=5, random_state=0)

In [12]:
%%time
y_pred = clf.predict(x_test_scaled)
# y_pred

CPU times: user 852 ms, sys: 96.5 ms, total: 948 ms
Wall time: 958 ms


In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test_scaled,y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.98      0.96    899500
         1.0       0.97      0.90      0.93    611800

    accuracy                           0.95   1511300
   macro avg       0.95      0.94      0.94   1511300
weighted avg       0.95      0.95      0.95   1511300



In [73]:
from sklearn.metrics import accuracy_score
print('Accuracy Score on train data: ', accuracy_score(y_true=y_train, y_pred=clf.predict(x_train)))
print('Accuracy Score on test data: ', accuracy_score(y_true=y_test, y_pred=clf.predict(x_test)))

Accuracy Score on train data:  0.991463898814588
Accuracy Score on test data:  0.9462052537550453


In [84]:
knn_classifier = NearestNeighbors(n_neighbors=5)

In [85]:
%%time
knn_classifier.fit(x_train_scaled, y_train_scaled)

CPU times: user 11.7 s, sys: 198 ms, total: 11.9 s
Wall time: 12.5 s


NearestNeighbors()

In [15]:
from sklearn.svm import SVC

#### Stopped SMV as it was taking a lot of time

In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = SVC()
clf.fit(x_train_scaled, y_train_scaled)

### For ppt, slides

In [27]:
pdf[(pdf['churn'] == 1) & (pdf['bill_avg'] > np.percentile(pdf['bill_avg'],95))].shape

(1096, 11)

In [28]:
pdf[(pdf['churn'] == 0) & (pdf['bill_avg'] > np.percentile(pdf['bill_avg'],95))].shape

(1400, 11)

In [31]:
pdf[(pdf['churn'] == 0)]

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15.0,1.0,0.0,11.95,25.0,0.14,0.0,8.4,2.3,0.0,0.0
500,56.0,1.0,1.0,11.94,32.0,1.38,0.0,69.4,4.0,0.0,0.0
1100,174.0,1.0,1.0,11.93,10.0,1.28,0.0,10.6,0.6,0.0,0.0
1500,288.0,1.0,1.0,11.93,18.0,1.80,0.0,1.0,0.0,0.0,0.0
1600,291.0,1.0,0.0,11.93,22.0,0.01,1.0,10.6,2.4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7157000,1673802.0,1.0,0.0,0.02,0.0,0.15,0.0,2.0,0.1,0.0,0.0
7157300,1673859.0,1.0,0.0,0.02,0.0,1.05,0.0,4.8,0.2,0.0,0.0
7158300,1674050.0,0.0,0.0,0.02,1.0,0.96,0.0,1.1,0.3,0.0,0.0
7158600,1674100.0,1.0,1.0,0.02,1.0,1.95,0.0,0.1,0.2,0.0,0.0
